
# Análisis de Calidad de Datos

Este notebook realiza un análisis de calidad de datos, comparando los resultados de **FTD** y **CPA** con los registros internos. 
Además, se analiza el histórico de depósitos y se identifican patrones clave en los datos.

## Pasos:
1. **Carga de datos**: Importación de archivos CSV.
2. **Comparación de FTD y CPA** con `internal_records.csv`.
3. **Detección de discrepancias** en los valores registrados.
4. **Visualización de discrepancias**.
5. **Análisis de depósitos**:
   - Histórico de dinero depositado.
   - Jugador que más ha depositado.
   - Otras visualizaciones clave.


In [ ]:

import pandas as pd
import matplotlib.pyplot as plt

# Cargar los datos desde los archivos CSV
internal_records = pd.read_csv("internal_records.csv")
players = pd.read_csv("players.csv")
traffic_sources = pd.read_csv("traffic_sources.csv")
deposits = pd.read_csv("deposits.csv")

# Convertir fechas a formato datetime
internal_records["month"] = pd.to_datetime(internal_records["month"])
deposits["deposit_date"] = pd.to_datetime(deposits["deposit_date"])
players["registration_date"] = pd.to_datetime(players["registration_date"])
traffic_sources["registration_date"] = pd.to_datetime(traffic_sources["registration_date"])

print("Datos cargados correctamente")



## Análisis de FTD y CPA

Aquí se comparan los valores esperados de **First Time Depositors (FTD)** y **Costo por Adquisición (CPA)** 
con los datos reales obtenidos a partir del dataset de jugadores y tráfico.


In [ ]:

# Contar FTD por mes
players["month"] = players["registration_date"].dt.to_period("M")
ftd_counts = players.groupby("month").size().reset_index(name="actual_ftd")

# Calcular CPA por mes
traffic_sources["month"] = traffic_sources["registration_date"].dt.to_period("M")
cpa_counts = traffic_sources.groupby("month").size().reset_index(name="actual_cpa")

# Unir los datos calculados con los registros internos
comparison = internal_records.copy()
comparison["month"] = comparison["month"].dt.to_period("M")
comparison = comparison.merge(ftd_counts, on="month", how="left").merge(cpa_counts, on="month", how="left")

# Rellenar valores NaN con 0
comparison.fillna(0, inplace=True)

# Calcular discrepancias
comparison["ftd_discrepancy"] = comparison["actual_ftd"] - comparison["expected_ftd"]
comparison["cpa_discrepancy"] = comparison["actual_cpa"] - comparison["expected_cpa"]

# Filtrar meses con discrepancias
discrepant_months = comparison[(comparison["ftd_discrepancy"] != 0) | (comparison["cpa_discrepancy"] != 0)]

# Mostrar los meses con discrepancias
discrepant_months



## Visualización de Discrepancias

Los siguientes gráficos muestran las discrepancias detectadas en **FTD** y **CPA**.


In [ ]:

# Gráfico de discrepancias en FTD y CPA
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.bar(discrepant_months["month"].astype(str), discrepant_months["ftd_discrepancy"], color='red', alpha=0.7)
plt.axhline(0, color='black', linewidth=1, linestyle='dashed')
plt.xticks(rotation=45)
plt.xlabel("Mes")
plt.ylabel("Diferencia en FTD")
plt.title("Discrepancias en First Time Depositors (FTD)")

plt.subplot(1, 2, 2)
plt.bar(discrepant_months["month"].astype(str), discrepant_months["cpa_discrepancy"], color='blue', alpha=0.7)
plt.axhline(0, color='black', linewidth=1, linestyle='dashed')
plt.xticks(rotation=45)
plt.xlabel("Mes")
plt.ylabel("Diferencia en CPA")
plt.title("Discrepancias en CPA")

plt.tight_layout()
plt.show()



## Análisis de Depósitos

Ahora analizamos el total de dinero depositado por mes y el jugador que más dinero ha depositado.


In [ ]:

# Convertir montos de depósito a tipo numérico
deposits["deposit_amount"] = pd.to_numeric(deposits["deposit_amount"], errors='coerce')

# Calcular total de depósitos por mes
deposits["month"] = deposits["deposit_date"].dt.to_period("M")
monthly_deposits = deposits.groupby("month")["deposit_amount"].sum().reset_index()

# Identificar al jugador que más ha depositado
top_player = deposits.groupby("player_id")["deposit_amount"].sum().idxmax()
top_player_amount = deposits.groupby("player_id")["deposit_amount"].sum().max()
top_player_name = players.loc[players["player_id"] == top_player, "name"].values[0]

print(f"Jugador con más depósitos: {top_player_name}, Total: ${top_player_amount}")

# Mostrar tabla de depósitos por mes
monthly_deposits



## Gráfico del Histórico de Depósitos

El siguiente gráfico muestra la evolución del dinero depositado por mes.


In [ ]:

# Gráfico del histórico de dinero depositado
plt.figure(figsize=(10, 5))
plt.plot(monthly_deposits["month"].astype(str), monthly_deposits["deposit_amount"], marker='o', linestyle='-')
plt.xticks(rotation=45)
plt.xlabel("Mes")
plt.ylabel("Total depositado ($)")
plt.title("Histórico de dinero depositado por mes")
plt.grid(True)
plt.show()
